<a href="https://colab.research.google.com/github/Coltcult/fantastic-computing-machine/blob/main/Simple_Interactive_Neural_Network_for_Forex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import datetime
import os

# --- Helper Functions ---

def load_forex_data(file_path, currency_pair, test_size=0.2, normalize=True, v_split=False):
    """
    Loads and preprocesses forex data from a CSV file.  Handles errors,
    supports train/test and train/validation/test splits, and normalization.

    Args:
        file_path (str): Path to the CSV file.
        currency_pair (str): Currency pair (for filename).
        test_size (float, optional): Proportion of data for testing. Defaults to 0.2.
        normalize (bool, optional): Whether to normalize the data. Defaults to True.
        v_split (bool, optional): Whether to create a validation set.

    Returns:
        tuple: (X_train, y_train, X_test, y_test) or
               (X_train, y_train, X_val, y_val, X_test, y_test) or
               (data_scaled, data)  on error
    """
    try:
        # 1. Load the data
        data = pd.read_csv(file_path)
        print(f"Loaded data from {file_path}")

        # Basic data inspection
        print(data.head())
        print(data.info())

        # 2. Data Cleaning and Feature Selection
        # Convert 'Date' to datetime (if it's not already)
        if 'Date' in data.columns:
            data['Date'] = pd.to_datetime(data['Date'])
        elif 'Time' in data.columns:  #check for Time instead of Date
            data['Time'] = pd.to_datetime(data['Time'])
        else:
            print("Error: Neither 'Date' nor 'Time' column found.  Ensure your CSV has a date or time column.")
            return None, None, None, None  # Error

        # Select relevant features.  Adjust this list as needed!
        # Common Forex features: Open, High, Low, Close, Volume
        features = ['Open', 'High', 'Low', 'Close', 'Volume']
        # Check if all features exist
        for feature in features:
            if feature not in data.columns:
                print(f"Error: Feature '{feature}' not found in the CSV file.")
                print(f"Available columns are: {data.columns.tolist()}")
                return None, None, None, None # Error

        data = data[['Date'] + features].copy() #keep date and the selected features.
        data = data.dropna()  # Drop rows with missing values
        if data.empty:
            print("Error: No data remaining after cleaning (e.g., all rows had missing values).")
            return None, None, None, None

        # 3. Prepare Target Variable (y) -  Predict 'Close' price change (simplified)
        data['Target'] = data['Close'].shift(-1) - data['Close']  # Predict next Close price change
        data = data.dropna()  # Remove last row (because of the shift)
        y = (data['Target'] > 0).astype(int)  # 1 if price goes up, 0 if down.  Simplified!


        # 4. Prepare Features (X)
        X = data[features].copy()  # Use selected features

        # 5. Normalize the data
        if normalize:
            scaler = MinMaxScaler()
            X = scaler.fit_transform(X)
            print("Data Normalized")


        # 6. Split data into training and testing sets
        if v_split:
          X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=test_size + 0.1, shuffle=False) #make test and val set.
          X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=test_size/(test_size + 0.1), shuffle=False)
          print(f"Data split into training ({len(X_train)}), validation({len(X_val)}), and testing ({len(X_test)}) sets.")
          return X_train, y_train, X_val, y_val, X_test, y_test

        else:
          X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, shuffle=False)
          print(f"Data split into training ({len(X_train)}) and testing ({len(X_test)}) sets.")
          return X_train, y_train, X_test, y_test

    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None, None, None, None  # Return None on error
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None, None, None # Return None on error

def create_model(input_shape, num_units=64, dropout_rate=0.2, optimizer='adam'):
    """
    Creates a simple neural network model for forex prediction.

    Args:
        input_shape (tuple): Shape of the input data.
        num_units (int, optional): Number of neurons in the dense layers. Defaults to 64.
        dropout_rate (float, optional): Dropout rate. Defaults to 0.2.
        optimizer (str, optional): Optimizer algorithm ('adam', 'sgd', 'rmsprop').
            Defaults to 'adam'.

    Returns:
        keras.Model: The compiled neural network model, or None on error.
    """
    try:
        model = keras.Sequential([
            keras.layers.Dense(num_units, activation='relu', input_shape=input_shape),
            keras.layers.Dropout(dropout_rate),
            keras.layers.Dense(num_units, activation='relu'),
            keras.layers.Dropout(dropout_rate),
            keras.layers.Dense(1, activation='sigmoid')  # Output: probability of price going up
        ])

        # Select optimizer
        if optimizer == 'adam':
            optimizer = 'adam'
        elif optimizer == 'sgd':
            optimizer = 'sgd'
        elif optimizer == 'rmsprop':
            optimizer = 'rmsprop'
        else:
            print(f"Error: Invalid optimizer '{optimizer}'.  Using 'adam' instead.")
            optimizer = 'adam'  # Default

        model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
        return model
    except Exception as e:
        print(f"Error creating model: {e}")
        return None

def train_model(model, X_train, y_train, X_val=None, y_val=None, epochs=100, batch_size=32,
                use_early_stopping=True, patience=10, verbose=1):
    """
    Trains the neural network model.

    Args:
        model (keras.Model): The neural network model to train.
        X_train (numpy.ndarray): Training data features.
        y_train (numpy.ndarray): Training data target.
        X_val (numpy.ndarray, optional): Validation data features.  If provided, enables early stopping.
        y_val (numpy.ndarray, optional): Validation data target.
        epochs (int, optional): Number of training epochs. Defaults to 100.
        batch_size (int, optional): Batch size. Defaults to 32.
        use_early_stopping (bool, optional): Whether to use early stopping. Defaults to True.
        patience (int, optional): Patience for early stopping. Defaults to 10.
        verbose (int, optional): Verbosity level (0, 1, or 2). Defaults to 1.

    Returns:
        tuple: (trained model, training history) or (None, None) on error
    """
    try:
        if use_early_stopping and X_val is not None and y_val is not None:
            early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience,
                                                            restore_best_weights=True, verbose=verbose)
            history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                                validation_data=(X_val, y_val), callbacks=[early_stopping],
                                verbose=verbose)
            return model, history
        else:
            history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                                verbose=verbose)
            return model, history
    except Exception as e:
        print(f"Error training model: {e}")
        return None, None

def evaluate_model(model, X_test, y_test, verbose=1):
    """
    Evaluates the trained model on the test set.

    Args:
        model (keras.Model): The trained neural network model.
        X_test (numpy.ndarray): Test data features.
        y_test (numpy.ndarray): Test data target.
        verbose (int): 0, 1 or 2.

    Returns:
        float: The test accuracy, or None on error.
    """
    try:
        _, accuracy = model.evaluate(X_test, y_test, verbose=verbose)
        return accuracy
    except Exception as e:
        print(f"Error evaluating model: {e}")
        return None

def plot_training_history(history, currency_pair=""):
    """
    Plots the training and validation loss and accuracy.

    Args:
        history (keras.callbacks.History): The training history object.
        currency_pair (str): currency pair
    """
    try:
        if history is None:
            print("No training history to plot.")
            return

        plt.figure(figsize=(12, 6))

        # Plot Loss
        plt.subplot(1, 2, 1)
        plt.plot(history.history['loss'], label='Training Loss')
        if 'val_loss' in history.history:
            plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title(f'{currency_pair} Model Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()

        # Plot Accuracy
        plt.subplot(1, 2, 2)
        plt.plot(history.history['accuracy'], label='Training Accuracy')
        if 'val_accuracy' in history.history:
            plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title(f'{currency_pair} Model Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.tight_layout()
        plt.show()
    except Exception as e:
        print(f"Error plotting training history: {e}")

def save_model(model, currency_pair):
    """
    Saves the trained model to a file.

    Args:
        model (keras.Model): The trained neural network model.
        currency_pair (str): The currency pair (used in filename).
    """
    try:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        model_filename = f'forex_model_{currency_pair}_{timestamp}.h5'
        model.save(model_filename)
        print(f"Saved model to {model_filename}")
        return model_filename
    except Exception as e:
        print(f"Error saving model: {e}")
        return None

def load_model(model_path):
    """Loads a saved model.

       Args:
          model_path(str): path to the model

       Returns:
          keras.Model: the loaded model or None on error
    """
    try:
        model = keras.models.load_model(model_path)
        print(f"Loaded model from {model_path}")
        return model
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

def predict(model, X_data):
    """Predicts the outcome for new data

    Args:
        model(keras.Model): the trained model
        X_data(np.ndarray): the input data

    Returns:
       np.ndarray: the predictions or None on error
    """
    try:
        predictions = model.predict(X_data)
        return predictions
    except Exception as e:
        print(f"Error making predictions: {e}")
        return None

# --- Main Function ---
def main():
    """
    Main function to orchestrate the forex prediction process.
    """
    print("Welcome to the Simple Forex Neural Network!")

    # 1. Get user input for the CSV file and currency pair
    file_path = input("Enter the path to your Forex CSV data file: ")
    currency_pair = input("Enter the currency pair (e.g., EURUSD): ")

    # 2. Load and preprocess the data
    use_validation_set = input("Use a validation set? (yes/no): ").lower() == 'yes'
    normalize_data = input("Normalize the data? (yes/no): ").lower() == 'yes'

    if use_validation_set:
        X_train, y_train, X_val, y_val, X_test, y_test = load_forex_data(file_path, currency_pair, v_split=True, normalize=normalize_data)
        if X_train is None:
            print("Failed to load and preprocess data. Exiting.")
            return
    else:
        X_train, y_train, X_test, y_test = load_forex_data(file_path, currency_pair, v_split=False, normalize=normalize_data)
        if X_train is None:
            print("Failed to load and preprocess data. Exiting.")
            return
        X_val = None
        y_val = None #set to None


    # 3. Create the neural network model
    input_shape = (X_train.shape[1],)
    num_units = int(input("Enter the number of neurons in hidden layers (e.g., 64): ") or 64)
    dropout_rate = float(input("Enter the dropout rate (e.g., 0.2): ") or 0.2)
    optimizer = input("Enter the optimizer (adam, sgd, rmsprop): ").lower() or 'adam' #let user choose the optimizer
    model = create_model(input_shape, num_units, dropout_rate, optimizer)
    if model is None:
        print("Failed to create model. Exiting.")
        return

    # 4. Train the model
    epochs = int(input("Enter the number of training epochs (e.g., 100): ") or 100)
    batch_size = int(input("Enter the batch size (e.g., 32): ") or 32)
    use_early_stopping = input("Use early stopping? (yes/no): ").lower() == 'yes'
    patience = int(input("Enter early stopping patience (e.g., 10): ") or 10)
    verbose = int(input("Enter verbosity level (0, 1, or 2): ") or 1) # let user choose verbosity

    model, history = train_model(model, X_train, y_train, X_val, y_val, epochs, batch_size,
                                    use_early_stopping, patience, verbose)
    if model is None:
        print("Failed to train model. Exiting.")
        return

    # 5. Evaluate the model
    accuracy = evaluate_model(model, X_test, y_test, verbose)
    if accuracy is None:
        print("Failed to evaluate model.")
    else:
        print(f"Test Accuracy: {accuracy:.4f}")

    # 6. Plot the training history
    plot_training_history(history, currency_pair)

    # 7. Save the model?
    save_model_choice = input("Save the trained model? (yes/no): ").lower()
    if save_model_choice == 'yes':
        saved_model_path = save_model(model, currency_pair)
        if saved_model_path:
            # 8. Load the model for prediction
            load_model_choice = input("Load the saved model for prediction? (yes/no): ").lower()
            if load_model_choice  == 'yes':
                loaded_model = load_model(saved_model_path)
                if loaded_model:
                    # 9. Make a prediction
                    # Get new data for prediction (in a real scenario, this would be live market data)
                    # For this example, we'll use the last few data points from the test set
                    num_predictions = int(input(f"Enter number of future predictions (max {len(X_test)}): ") or 5)
                    if num_predictions > len(X_test):
                         num_predictions = len(X_test)
                    X_predict = X_test[-num_predictions:]

                    predictions = predict(loaded_model, X_predict)
                    if predictions is not None:
                        print("\nPredictions:")
                        for i, prediction in enumerate(predictions):
                            # Convert prediction to "Up" or "Down" for easier interpretation
                            predicted_direction = "Up" if prediction[0] > 0.5 else "Down"
                            # Get the actual date for the prediction
                            predicted_date = data.iloc[-(num_predictions - i)]['Date'] if 'Date' in data.columns else data.iloc[-(num_predictions - i)]['Time']
                            print(f"  {predicted_date}: {predicted_direction} ({prediction[0]:.4f})")
                    else:
                        print("Failed to make predictions.")
                else:
                    print("Failed to load the model.")
    print("End")

if __name__ == "__main__":
    main()

"""
    Forex Neural Network User Guide and Instructions:

    1.  Purpose:
        This Python script implements a simple neural network designed to analyze Forex (foreign exchange) market data.  It's intended to help you explore how neural networks can be used to predict future price movements (whether a currency pair's price will go up or down).  **It is crucial to understand that this is a simplified example for educational purposes.** It is NOT intended for use in real-world trading without extensive modification, testing, and a thorough understanding of the risks involved.

    2.  Disclaimer:
        -   **This is NOT a trading system.** The script provides a basic framework for analysis.  It does not guarantee profits, and you could lose money if you use it for actual trading.
        -   **Forex trading is risky.** You can lose money rapidly due to factors like market volatility, leverage, and unforeseen economic events.
        -   **Use at your own risk.** The author of this code is not responsible for any financial losses you may incur.  You should consult with a qualified financial advisor before making any trading decisions.
        -   **This code is for educational purposes only.** Experiment with it, learn from it, but do not rely on it for real-world trading.

    3.  How This Code Can Be Useful (in an educational context):
        -   **Understanding Neural Networks:** The script demonstrates how a basic neural network is structured and trained.  You can see how data is loaded, preprocessed, fed into the network, and used to make predictions.
        -   **Exploring Forex Data:** You can use this script to analyze historical Forex data, identify patterns, and see how different features (Open, High, Low, Close, Volume) relate to price movements.
        -   **Experimenting with Parameters:** The script allows you to experiment with various parameters, such as the number of neurons, dropout rate, training epochs, and optimizer.  You can observe how these parameters affect the model's performance.
        -   **Learning about Machine Learning for Finance:** This script provides a starting point for learning how machine learning techniques can be applied to financial data.  It can help you understand the challenges and limitations of using machine learning in this domain.

    4.  System Requirements:
        -   Python 3.x
        -   TensorFlow (for neural networks)
        -   scikit-learn (for data splitting and scaling)
        -   pandas (for data manipulation)
        -   matplotlib (for plotting)
        -   NumPy
        -   PyPDF2 (to read PDF files) - will be installed if missing
        -   Google Colab (recommended) or a local Python environment with the above libraries installed.

    5.  Installation (Google Colab - Recommended):
        -   Open a Google Colab notebook in your browser.
        -   Copy and paste the code into a cell in the notebook.
        -   Run the cell. Colab will automatically install the necessary libraries.
        -   Upload your Forex CSV data file to the Colab environment.

    6.  Installation (Local Python Environment):
        -   Install Python 3.x on your system.
        -   Install the required libraries using pip:
            ```bash
            pip install tensorflow scikit-learn pandas matplotlib numpy

SyntaxError: incomplete input (<ipython-input-1-c0859b03417d>, line 386)

-   Save the code as a Python file (e.g., `forex_neural_network.py`).
        -   Run the script from your terminal:
            ```bash
            python forex_neural_network.py
            ```

    7.  Data Preparation:
        -   **Data Source:** You will need historical Forex data in CSV format.  Reliable sources include Dukascopy, TrueFX, and your broker (if they provide historical data).
        -   **Data Format:** The CSV file should contain columns for:
            -   `Date` or `Time`:  The date and time of each data point.  The code will automatically detect either 'Date' or 'Time'.
            -   `Open`:  The opening price for the period.
            -   `High`:  The highest price for the period.
            -   `Low`:  The lowest price for the period.
            -   `Close`:  The closing price for the period.
            -   `Volume`:  The trading volume for the period.
        -   **Data Cleaning:** Ensure your data is relatively clean (missing values handled, consistent formatting).  The script does some basic cleaning (dropping rows with missing values), but you may need to do more preprocessing depending on your data source.
        -    **Timeframe:** The data should be in a consistent timeframe (e.g., 1-hour, 1-day, etc.).

    8.  How to Use the Code:
        -   Run the script.
        -   The script will prompt you to enter the following:
            -   **CSV File Path:** Enter the path to your Forex CSV data file (e.g., `EURUSD_data.csv`).
            -   **Currency Pair:** Enter the currency pair you are analyzing (e.g., `EURUSD`, `GBPUSD`, `USDJPY`).  This is used for naming the saved model.
            -   **Use a validation set?:** Enter 'yes' to split the training data into training and validation sets. This is recommended for better model training.
            -   **Normalize the data?:** Enter 'yes' to normalize the input data.  This is generally recommended for neural networks.
            -   **Number of Neurons:** Enter the number of neurons in the hidden layers of the neural network (e.g., 64, 128, 256).  More neurons can capture more complex patterns but may also lead to overfitting.
            -   **Dropout Rate:** Enter the dropout rate (e.g., 0.2, 0.3, 0.5).  Dropout is a regularization technique that helps prevent overfitting.  A common range is 0.1 to 0.5.
            -   **Optimizer:** Choose an optimization algorithm for training the neural network ('adam', 'sgd', or 'rmsprop').  Adam is often a good default choice.
            -   **Number of Epochs:** Enter the number of training epochs (e.g., 100, 200, 500).  An epoch is one complete pass through the training data.
            -   **Batch Size:** Enter the batch size (e.g., 32, 64, 128).  The number of data points processed before the model's weights are updated.
            -   **Use early stopping?:** Enter 'yes' to use early stopping.  This will stop the training process if the model's performance on the validation set stops improving, preventing overfitting.
            -   **Early stopping patience:** Enter the number of epochs to wait for improvement in validation loss before stopping.
            -   **Verbosity level:** 0, 1, or 2.  0 = silent, 1 = progress bar, 2 = one line per epoch.
            -   **Save the trained model?:** Enter 'yes' to save the trained neural network model to a file.  This allows you to load and reuse the model later without retraining.
            -   **Load the saved model for prediction?:** If you saved a model, you can choose to load it and make a prediction on some data
            -   **Number of future predictions?:** Enter how many future predictions you want to make
        -   The script will then:
            -   Load and preprocess the data from your CSV file.
            -   Create and train the neural network model.
            -   Evaluate the model's performance on the test data.
            -   Display plots of the training and validation loss and accuracy.
            -   Save the trained model (if you choose to).
            -   Make predictions (if you choose to).

    9.  Understanding the Output:
        -   **Test Accuracy:** This is the percentage of times the model correctly
Okay, I've added detailed instructions and a user guide directly into the code as comments. This should provide comprehensive guidance on how to use the script and clarify its purpose for forex trading analysis.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import datetime
import os

# --- Helper Functions ---

def load_forex_data(file_path, currency_pair, test_size=0.2, normalize=True, v_split=False):
    """
    Loads and preprocesses forex data from a CSV file.  Handles errors,
    supports train/test and train/validation/test splits, and normalization.

    Args:
        file_path (str): Path to the CSV file.
        currency_pair (str): Currency pair (for filename).
        test_size (float, optional): Proportion of data for testing. Defaults to 0.2.
        normalize (bool, optional): Whether to normalize the data. Defaults to True.
        v_split (bool, optional): Whether to create a validation set.

    Returns:
        tuple: (X_train, y_train, X_test, y_test) or
               (X_train, y_train, X_val, y_val, X_test, y_test) or
               (data_scaled, data)  on error
    """
    try:
        # 1. Load the data
        data = pd.read_csv(file_path)
        print(f"Loaded data from {file_path}")

        # Basic data inspection
        print(data.head())
        print(data.info())

        # 2. Data Cleaning and Feature Selection
        # Convert 'Date' to datetime (if it's not already)
        if 'Date' in data.columns:
            data['Date'] = pd.to_datetime(data['Date'])
        elif 'Time' in data.columns:  #check for Time instead of Date
            data['Time'] = pd.to_datetime(data['Time'])
        else:
            print("Error: Neither 'Date' nor 'Time' column found.  Ensure your CSV has a date or time column.")
            return None, None, None, None  # Error

        # Select relevant features.  Adjust this list as needed!
        # Common Forex features: Open, High, Low, Close, Volume
        features = ['Open', 'High', 'Low', 'Close', 'Volume']
        # Check if all features exist
        for feature in features:
            if feature not in data.columns:
                print(f"Error: Feature '{feature}' not found in the CSV file.")
                print(f"Available columns are: {data.columns.tolist()}")
                return None, None, None, None # Error

        data = data[['Date'] + features].copy() #keep date and the selected features.
        data = data.dropna()  # Drop rows with missing values
        if data.empty:
            print("Error: No data remaining after cleaning (e.g., all rows had missing values).")
            return None, None, None, None

        # 3. Prepare Target Variable (y) -  Predict 'Close' price change (simplified)
        data['Target'] = data['Close'].shift(-1) - data['Close']  # Predict next Close price change
        data = data.dropna()  # Remove last row (because of the shift)
        y = (data['Target'] > 0).astype(int)  # 1 if price goes up, 0 if down.  Simplified!


        # 4. Prepare Features (X)
        X = data[features].copy()  # Use selected features

        # 5. Normalize the data
        if normalize:
            scaler = MinMaxScaler()
            X = scaler.fit_transform(X)
            print("Data Normalized")


        # 6. Split data into training and testing sets
        if v_split:
          X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=test_size + 0.1, shuffle=False) #make test and val set.
          X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=test_size/(test_size + 0.1), shuffle=False)
          print(f"Data split into training ({len(X_train)}), validation({len(X_val)}), and testing ({len(X_test)}) sets.")
          return X_train, y_train, X_val, y_val, X_test, y_test

        else:
          X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, shuffle=False)
          print(f"Data split into training ({len(X_train)}) and testing ({len(X_test)}) sets.")
          return X_train, y_train, X_test, y_test

    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None, None, None, None  # Return None on error
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None, None, None # Return None on error

def create_model(input_shape, num_units=64, dropout_rate=0.2, optimizer='adam'):
    """
    Creates a simple neural network model for forex prediction.

    Args:
        input_shape (tuple): Shape of the input data.
        num_units (int, optional): Number of neurons in the dense layers. Defaults to 64.
        dropout_rate (float, optional): Dropout rate. Defaults to 0.2.
        optimizer (str, optional): Optimizer algorithm ('adam', 'sgd', 'rmsprop').
            Defaults to 'adam'.

    Returns:
        keras.Model: The compiled neural network model, or None on error.
    """
    try:
        model = keras.Sequential([
            keras.layers.Dense(num_units, activation='relu', input_shape=input_shape),
            keras.layers.Dropout(dropout_rate),
            keras.layers.Dense(num_units, activation='relu'),
            keras.layers.Dropout(dropout_rate),
            keras.layers.Dense(1, activation='sigmoid')  # Output: probability of price going up
        ])

        # Select optimizer
        if optimizer == 'adam':
            optimizer = 'adam'
        elif optimizer == 'sgd':
            optimizer = 'sgd'
        elif optimizer == 'rmsprop':
            optimizer = 'rmsprop'
        else:
            print(f"Error: Invalid optimizer '{optimizer}'.  Using 'adam' instead.")
            optimizer = 'adam'  # Default

        model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
        return model
    except Exception as e:
        print(f"Error creating model: {e}")
        return None

def train_model(model, X_train, y_train, X_val=None, y_val=None, epochs=100, batch_size=32,
                use_early_stopping=True, patience=10, verbose=1):
    """
    Trains the neural network model.

    Args:
        model (keras.Model): The neural network model to train.
        X_train (numpy.ndarray): Training data features.
        y_train (numpy.ndarray): Training data target.
        X_val (numpy.ndarray, optional): Validation data features.  If provided, enables early stopping.
        y_val (numpy.ndarray, optional): Validation data target.
        epochs (int, optional): Number of training epochs. Defaults to 100.
        batch_size (int, optional): Batch size. Defaults to 32.
        use_early_stopping (bool, optional): Whether to use early stopping. Defaults to True.
        patience (int, optional): Patience for early stopping. Defaults to 10.
        verbose (int, optional): Verbosity level (0, 1, or 2). Defaults to 1.

    Returns:
        tuple: (trained model, training history) or (None, None) on error
    """
    try:
        if use_early_stopping and X_val is not None and y_val is not None:
            early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience,
                                                            restore_best_weights=True, verbose=verbose)
            history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                                validation_data=(X_val, y_val), callbacks=[early_stopping],
                                verbose=verbose)
            return model, history
        else:
            history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                                verbose=verbose)
            return model, history
    except Exception as e:
        print(f"Error training model: {e}")
        return None, None

def evaluate_model(model, X_test, y_test, verbose=1):
    """
    Evaluates the trained model on the test set.

    Args:
        model (keras.Model): The trained neural network model.
        X_test (numpy.ndarray): Test data features.
        y_test (numpy.ndarray): Test data target.
        verbose (int): 0, 1 or 2.

    Returns:
        float: The test accuracy, or None on error.
    """
    try:
        _, accuracy = model.evaluate(X_test, y_test, verbose=verbose)
        return accuracy
    except Exception as e:
        print(f"Error evaluating model: {e}")
        return None

def plot_training_history(history, currency_pair=""):
    """
    Plots the training and validation loss and accuracy.

    Args:
        history (keras.callbacks.History): The training history object.
        currency_pair (str): currency pair
    """
    try:
        if history is None:
            print("No training history to plot.")
            return

        plt.figure(figsize=(12, 6))

        # Plot Loss
        plt.subplot(1, 2, 1)
        plt.plot(history.history['loss'], label='Training Loss')
        if 'val_loss' in history.history:
            plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title(f'{currency_pair} Model Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()

        # Plot Accuracy
        plt.subplot(1, 2, 2)
        plt.plot(history.history['accuracy'], label='Training Accuracy')
        if 'val_accuracy' in history.history:
            plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title(f'{currency_pair} Model Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.tight_layout()
        plt.show()
    except Exception as e:
        print(f"Error plotting training history: {e}")

def save_model(model, currency_pair):
    """
    Saves the trained model to a file.

    Args:
        model (keras.Model): The trained neural network model.
        currency_pair (str): The currency pair (used in filename).
    """
    try:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        model_filename = f'forex_model_{currency_pair}_{timestamp}.h5'
        model.save(model_filename)
        print(f"Saved model to {model_filename}")
        return model_filename
    except Exception as e:
        print(f"Error saving model: {e}")
        return None

def load_model(model_path):
    """Loads a saved model.

       Args:
          model_path(str): path to the model

       Returns:
          keras.Model: the loaded model or None on error
    """
    try:
        model = keras.models.load_model(model_path)
        print(f"Loaded model from {model_path}")
        return model
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

def predict(model, X_data):
    """Predicts the outcome for new data

    Args:
        model(keras.Model): the trained model
        X_data(np.ndarray): the input data

    Returns:
       np.ndarray: the predictions or None on error
    """
    try:
        predictions = model.predict(X_data)
        return predictions
    except Exception as e:
        print(f"Error making predictions: {e}")
        return None

# --- Main Function ---
def main():
    """
    Main function to orchestrate the forex prediction process.
    """
    print("Welcome to the Simple Forex Neural Network!")

    # 1. Get user input for the CSV file and currency pair
    file_path = input("Enter the path to your Forex CSV data file: ")
    currency_pair = input("Enter the currency pair (e.g., EURUSD): ")

    # 2. Load and preprocess the data
    use_validation_set = input("Use a validation set? (yes/no): ").lower() == 'yes'
    normalize_data = input("Normalize the data? (yes/no): ").lower() == 'yes'

    if use_validation_set:
        X_train, y_train, X_val, y_val, X_test, y_test = load_forex_data(file_path, currency_pair, v_split=True, normalize=normalize_data)
        if X_train is None:
            print("Failed to load and preprocess data. Exiting.")
            return
    else:
        X_train, y_train, X_test, y_test = load_forex_data(file_path, currency_pair, v_split=False, normalize=normalize_data)
        if X_train is None:
            print("Failed to load and preprocess data. Exiting.")
            return
        X_val = None
        y_val = None #set to None


    # 3. Create the neural network model
    input_shape = (X_train.shape[1],)
    num_units = int(input("Enter the number of neurons in hidden layers (e.g., 64): ") or 64)
    dropout_rate = float(input("Enter the dropout rate (e.g., 0.2): ") or 0.2)
    optimizer = input("Enter the optimizer (adam, sgd, rmsprop): ").lower() or 'adam' #let user choose the optimizer
    model = create_model(input_shape, num_units, dropout_rate, optimizer)
    if model is None:
        print("Failed to create model. Exiting.")
        return

    # 4. Train the model
    epochs = int(input("Enter the number of training epochs (e.g., 100): ") or 100)
    batch_size = int(input("Enter the batch size (e.g., 32): ") or 32)
    use_early_stopping = input("Use early stopping? (yes/no): ").lower() == 'yes'
    patience = int(input("Enter early stopping patience (e.g., 10): ") or 10)
    verbose = int(input("Enter verbosity level (0, 1, or 2): ") or 1) # let user choose verbosity

    model, history = train_model(model, X_train, y_train, X_val, y_val, epochs, batch_size,
                                    use_early_stopping, patience, verbose)
    if model is None:
        print("Failed to train model. Exiting.")
        return

    # 5. Evaluate the model
    accuracy = evaluate_model(model, X_test, y_test, verbose)
    if accuracy is None:
        print("Failed to evaluate model.")
    else:
        print(f"Test Accuracy: {accuracy:.4f}")

    # 6. Plot the training history
    plot_training_history(history, currency_pair)

    # 7. Save the model?
    save_model_choice = input("Save the trained model? (yes/no): ").lower()
    if save_model_choice == 'yes':
        saved_model_path = save_model(model, currency_pair)
        if saved_model_path:
            # 8. Load the model for prediction
            load_model_choice = input("Load the saved model for prediction? (yes/no): ").lower()
            if load_model_choice  == 'yes':
                loaded_model = load_model(saved_model_path)
                if loaded_model:
                    # 9. Make a prediction
                    # Get new data for prediction (in a real scenario, this would be live market data)
                    # For this example, we'll use the last few data points from the test set
                    num_predictions = int(input(f"Enter number of future predictions (max {len(X_test)}): ") or 5)
                    if num_predictions > len(X_test):
                         num_predictions = len(X_test)
                    X_predict = X_test[-num_predictions:]

                    predictions = predict(loaded_model, X_predict)
                    if predictions is not None:
                        print("\nPredictions:")
                        for i, prediction in enumerate(predictions):
                            # Convert prediction to "Up" or "Down" for easier interpretation
                            predicted_direction = "Up" if prediction[0] > 0.5 else "Down"
                            # Get the actual date for the prediction
                            predicted_date = data.iloc[-(num_predictions - i)]['Date'] if 'Date' in data.columns else data.iloc[-(num_predictions - i)]['Time']
                            print(f"  {predicted_date}: {predicted_direction} ({prediction[0]:.4f})")
                    else:
                        print("Failed to make predictions.")
                else:
                    print("Failed to load the model.")
    print("End")

if __name__ == "__main__":
    main()

"""
    Forex Neural Network User Guide and Instructions:

    1.  Purpose:
        This Python script implements a simple neural network designed to analyze Forex (foreign exchange) market data.  It's intended to help you explore how neural networks can be used to predict future price movements (whether a currency pair's price will go up or down).  **It is crucial to understand that this is a simplified example for educational purposes.** It is NOT intended for use in real-world trading without extensive modification, testing, and a thorough understanding of the risks involved.

    2.  Disclaimer:
        -   **This is NOT a trading system.** The script provides a basic framework for analysis.  It does not guarantee profits, and you could lose money if you use it for actual trading.
        -   **Forex trading is risky.** You can lose money rapidly due to factors like market volatility, leverage, and unforeseen economic events.
        -   **Use at your own risk.** The author of this code is not responsible for any financial losses you may incur.  You should consult with a qualified financial advisor before making any trading decisions.
        -   **This code is for educational purposes only.** Experiment with it, learn from it, but do not rely on it for real-world trading.

    3.  How This Code Can Be Useful (in an educational context):
        -   **Understanding Neural Networks:** The script demonstrates how a basic neural network is structured and trained.  You can see how data is loaded, preprocessed, fed into the network, and used to make predictions.
        -   **Exploring Forex Data:** You can use this script to analyze historical Forex data, identify patterns, and see how different features (Open, High, Low, Close, Volume) relate to price movements.
        -   **Experimenting with Parameters:** The script allows you to experiment with various parameters, such as the number of neurons, dropout rate, training epochs, and optimizer.  You can observe how these parameters affect the model's performance.
        -   **Learning about Machine Learning for Finance:** This script provides a starting point for learning how machine learning techniques can be applied to financial data.  It can help you understand the challenges and limitations of using machine learning in this domain.

    4.  System Requirements:
        -   Python 3.x
        -   TensorFlow (for neural networks)
        -   scikit-learn (for data splitting and scaling)
        -   pandas (for data manipulation)
        -   matplotlib (for plotting)
        -   NumPy
        -   PyPDF2 (to read PDF files) - will be installed if missing
        -   Google Colab (recommended) or a local Python environment with the above libraries installed.

    5.  Installation (Google Colab - Recommended):
        -   Open a Google Colab notebook in your browser.
        -   Copy and paste the code into a cell in the notebook.
        -   Run the cell. Colab will automatically install the necessary libraries.
        -   Upload your Forex CSV data file to the Colab environment.

    6.  Installation (Local Python Environment):
        -   Install Python 3.x on your system.
        -   Install the required libraries using pip:
            ```bash
            pip install tensorflow scikit-learn pandas matplotlib numpy

-   Save the code as a Python file (e.g., `forex_neural_network.py`).
        -   Run the script from your terminal:
            ```bash
            python forex_neural_network.py
            ```

    7.  Data Preparation:
        -   **Data Source:** You will need historical Forex data in CSV format.  Reliable sources include Dukascopy, TrueFX, and your broker (if they provide historical data).
        -   **Data Format:** The CSV file should contain columns for:
            -   `Date` or `Time`:  The date and time of each data point.  The code will automatically detect either 'Date' or 'Time'.
            -   `Open`:  The opening price for the period.
            -   `High`:  The highest price for the period.
            -   `Low`:  The lowest price for the period.
            -   `Close`:  The closing price for the period.
            -   `Volume`:  The trading volume for the period.
        -   **Data Cleaning:** Ensure your data is relatively clean (missing values handled, consistent formatting).  The script does some basic cleaning (dropping rows with missing values), but you may need to do more preprocessing depending on your data source.
        -    **Timeframe:** The data should be in a consistent timeframe (e.g., 1-hour, 1-day, etc.).

    8.  How to Use the Code:
        -   Run the script.
        -   The script will prompt you to enter the following:
            -   **CSV File Path:** Enter the path to your Forex CSV data file (e.g., `EURUSD_data.csv`).
            -   **Currency Pair:** Enter the currency pair you are analyzing (e.g., `EURUSD`, `GBPUSD`, `USDJPY`).  This is used for naming the saved model.
            -   **Use a validation set?:** Enter 'yes' to split the training data into training and validation sets. This is recommended for better model training.
            -   **Normalize the data?:** Enter 'yes' to normalize the input data.  This is generally recommended for neural networks.
            -   **Number of Neurons:** Enter the number of neurons in the hidden layers of the neural network (e.g., 64, 128, 256).  More neurons can capture more complex patterns but may also lead to overfitting.
            -   **Dropout Rate:** Enter the dropout rate (e.g., 0.2, 0.3, 0.5).  Dropout is a regularization technique that helps prevent overfitting.  A common range is 0.1 to 0.5.
            -   **Optimizer:** Choose an optimization algorithm for training the neural network ('adam', 'sgd', or 'rmsprop').  Adam is often a good default choice.
            -   **Number of Epochs:** Enter the number of training epochs (e.g., 100, 200, 500).  An epoch is one complete pass through the training data.
            -   **Batch Size:** Enter the batch size (e.g., 32, 64, 128).  The number of data points processed before the model's weights are updated.
            -   **Use early stopping?:** Enter 'yes' to use early stopping.  This will stop the training process if the model's performance on the validation set stops improving, preventing overfitting.
            -   **Early stopping patience:** Enter the number of epochs to wait for improvement in validation loss before stopping.
            -   **Verbosity level:** 0, 1, or 2.  0 = silent, 1 = progress bar, 2 = one line per epoch.
            -   **Save the trained model?:** Enter 'yes' to save the trained neural network model to a file.  This allows you to load and reuse the model later without retraining.
            -   **Load the saved model for prediction?:** If you saved a model, you can choose to load it and make a prediction on some data
            -   **Number of future predictions?:** Enter how many future predictions you want to make
        -   The script will then:
            -   Load and preprocess the data from your CSV file.
            -   Create and train the neural network model.
            -   Evaluate the model's performance on the test data.
            -   Display plots of the training and validation loss and accuracy.
            -   Save the trained model (if you choose to).
            -   Make predictions (if you choose to).

    9.  Understanding the Output:
        -   **Test Accuracy:** This is the percentage of times the model correc
  ```